# Modify the intensity of TC scenarios



In [ ]:
%matplotlib inline

import os
from os.path import join as pjoin
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.collections import LineCollection
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.lines import Line2D

import cartopy.crs as ccrs
import cartopy.feature as feature
import numpy as np
from Utilities.track import ncReadTrackData, ncSaveTracks
from Utilities.loadData import maxWindSpeed

import pandas as pd
from datetime import datetime

import seaborn as sns
sns.set_context('talk')

In [ ]:
def makeSegments(xx, yy):
    points = np.array([xx, yy]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    return segments

def colorline(ax, xdata, ydata, zdata=None, alpha=0.9):
    """
    Given a collection of x,y points and optionally magnitude
    values for each point, plot the data as a collection of
    coloured line segments. Line segments are added to the given 
    :class:`matplotlib.axes` instance.
    
    .. note:: Currently, intervals are hard-coded
              for plotting the central pressure of TCs. 
              [800, 920, 935, 950, 970, 985, 1050]
    
    :params ax: :class:`matplotlib.axes` instance on which to plot the line segments
    :param xdata: array of x-coordinates of points to plot
    :param ydata: array of y-coordinates of points to plot
    :param zdata: (optional) array of magnitude values of the points to inform colouring
    :param alpha: transparency of the lines (default=0.9)
    
    """
    colours=['0.75', '#0FABF6', '#0000FF', 
             '#00FF00', '#FF8100', '#ff0000']
    intervals = [0, 17.5, 24.5, 32.5, 44.2, 55.5, 1000]
    intervals = [0, 25, 35, 46, 62, 77, 200]
    #intervals = [800, 920, 935, 950, 970, 985, 1050]
    segments = makeSegments(xdata, ydata)
    cmap = ListedColormap(colours)
    norm = BoundaryNorm(intervals, cmap.N)
    lc = LineCollection(segments, array=zdata, cmap=cmap,
                        norm=norm, alpha=alpha)

    labels = ['No data', 'Category 1', 'Category 2',
              'Category 3', 'Category 4', 'Category 5']
    handles = []
    for c, l in zip(cmap.colors, labels):
        handles.append(Line2D([0], [0], color=c, label=l))

    ax.add_collection(lc)
    ax.legend(handles, labels, loc=2, frameon=True, prop={'size': 10})
    return ax

In [ ]:
dataPath = r"\\prod.lan\active\ops\community_safety\georisk\HaRIA_B_Wind\projects\qfes_swha\data\derived\tracks\seq"
source_track = pjoin(dataPath, 'track.004-08495.nc')
tracks = ncReadTrackData(source_track)
t = pd.DataFrame.from_records(tracks[0].data)
t.Datetime = t.Datetime.apply(lambda x: datetime.strptime(x.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S"))
dtname = 'Datetime'
idx = t.CycloneNumber.values
cp = t.CentralPressure.values

varidx = np.ones(len(idx))
varidx[1:][idx[1:]==idx[:-1]] = 0
dt = (t[dtname] - t[dtname].shift()).fillna(pd.Timedelta(seconds=0)).apply(lambda x: x / np.timedelta64(1,'m')).astype('int64') % (24*60) / 60

vmax = maxWindSpeed(varidx, dt.values, t.Longitude.values, t.Latitude.values,
                              t.CentralPressure.values, t.EnvPressure.values, gustfactor=1.223)

In [ ]:
t.to_csv(pjoin(dataPath, 'track.004-08495.csv'))

Make modifications to the track data in Excel or similar to make the process easier

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 6))
ax.plot(t.Datetime, vmax, color='k', label="Base scenario")

plt.xticks(rotation='vertical')
#[0, 25, 35, 46, 62, 77, 200]
ax.axhline(25, color='#0FABF6')
ax.axhline(35, color='#0000FF')
ax.axhline(46, color='#00FF00')
ax.axhline(62, color='#FF8100')
ax.axhline(77, color='#FF0000')
ax.grid(True)
ax.set_ylabel("Wind speed (m/s)")
ax.set_xlabel("Time")
ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
#plt.savefig(r"\\prod.lan\active\ops\community_safety\georisk\HaRIA_B_Wind\projects\powerlink\data\derived\tracks\007-02914_scaling.png", bbox_inches="tight")
None

In [ ]:
source_track_mod = pjoin(dataPath, 'track.004-08495b.csv')
tm = pd.read_csv(source_track_mod, parse_dates=[2], infer_datetime_format=True)
#tm['Datetime'] = tm.Datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M"))
#tm['Datetime'] = tm.Datetime.dt.to_pydatetime()

#tm = pd.DataFrame.from_records(tracks_mod[0].data)

In [ ]:
newtm = tm.astype({'Datetime':"O"}).drop("Unnamed: 0", axis=1)

In [ ]:
newtm

In [ ]:
newdtypes = tm.dtypes.to_dict()

In [ ]:
mydtypes = [
 ('CycloneNumber', 'int64'),
 ('Datetime', datetime),
 ('TimeElapsed', 'int64'),
 ('Longitude', 'float64'),
 ('Latitude', 'float64'),
 ('Speed', 'float64'),
 ('Bearing', 'float64'),
 ('CentralPressure', 'float64'),
 ('EnvPressure', 'float64'),
 ('rMax', 'float64')]

In [ ]:
newdtypes['Datetime'] = np.dtype('O')

In [ ]:
tm['Datetime'] = tm.Datetime.dt.to_pydatetime()
#apply(lambda x: pd.to_datetime(x))

In [ ]:
type(tm.Datetime[0])

In [ ]:
def scalePressureDeficit(pc, poci, scale=0.8):
    scaleddp = scale * (poci - pc)
    return poci - scaleddp

In [ ]:
varidxm = np.zeros(len(tm.Datetime))
varidx[0] = 1
dtm = (tm[dtname] - tm[dtname].shift()).fillna(pd.Timedelta(seconds=0)).apply(lambda x: x / np.timedelta64(1,'m')).astype('int64') % (24*60) / 60

vmaxm = maxWindSpeed(varidxm, dtm.values, tm.Longitude.values, tm.Latitude.values,
                              tm.CentralPressure.values, tm.EnvPressure.values, gustfactor=1.223)


In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='10m', color='black', linewidth=1)
ax.add_feature(feature.BORDERS)
gl = ax.gridlines(linestyle=":", draw_labels=True)
ax.add_feature(feature.LAND, zorder=0)
ax.set_xlim((147.5, 157.5))
ax.set_ylim((-32, -20))
colorline(ax, t.Longitude, t.Latitude, vmaxm)
#None

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 6))
ax.plot(t.Datetime, vmax, color='k', label="Base scenario")

ax.plot(tm.Datetime, vmaxm, color='gray', label=r"1.7 $\Delta p$")

plt.xticks(rotation='vertical')
#[0, 25, 35, 46, 62, 77, 200]
ax.axhline(25, color='#0FABF6')
ax.axhline(35, color='#0000FF')
ax.axhline(46, color='#00FF00')
ax.axhline(62, color='#FF8100')
ax.axhline(77, color='#FF0000')
ax.grid(True)
ax.set_ylabel("Wind speed (m/s)")
ax.set_xlabel("Time")
ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
#plt.savefig(r"\\prod.lan\active\ops\community_safety\georisk\HaRIA_B_Wind\projects\powerlink\data\derived\tracks\007-02914_scaling.png", bbox_inches="tight")
None

In [ ]:
tracks[0].data = tm.drop("Unnamed: 0", axis=1).to_records(index=False)
tracks[0].data.dtypes = mydtypes

#nndt = [datetime.utcfromtimestamp(tracks[0].Datetime.astype(datetime)[i]/1e9) for i in range(len(tracks[0].Datetime))]

newtrackdata = np.recarray(tracks[0].data.shape, dtype=mydtypes)

In [ ]:
newtrackdata['Datetime'] = tm['Datetime']

for col in ['CycloneNumber', 'TimeElapsed', 'Longitude',
       'Latitude', 'Speed', 'Bearing', 'CentralPressure', 'EnvPressure',
       'rMax']:
    newtrackdata[col] = tracks[0].data[col]

tracks[0].data = newtrackdata

In [ ]:
datetime.strptime(datetime.strftime(tracks[0].Datetime[0], "%Y-%m-%d %H:%M"), "%Y-%m-%d %H:%M")

In [ ]:
tm['Datetime'] = tm.Datetime.apply(lambda x: datetime.strptime(datetime.strftime(x, "%Y-%m-%d %H:%M"), "%Y-%m-%d %H:%M"))

In [ ]:
tm.Datetime

In [ ]:
scenarioTrackFile = pjoin(dataPath, 'track.004-08495b.nc')
atts = {"history":"Linear decline in intensity from base scenario",
        "title":"Synthetic tropical cyclone track scenario 004-08495b"}
ncSaveTracks(scenarioTrackFile, tracks, calendar='julian', attributes=atts)